<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js110_electronPQM2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js110_electronPQM2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: http://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.12.12 created,  last updated on 2025.01.05
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js110_electronPQM2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js110_electronPQM2D
//    Copyright(C) 2019-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2019.03.21 created, last updated on 2020.12.15
//    ver 0.0.1  2021,10,12 v1, last updated on 2021.10.12
//    ver 0.0.2  2021.11.07 v2, last updated on 2021.11.07
//    ver 0.0.3  2023.05.17 v3, last updated on 2023.09.12
//
//
// --------------------  PQM: Periodic Quantum Mechanics 2D (explecit method)
//
// - time dependent Schrodinger equation: i(d/dt)psi(r,t) = H psi(r,t)
//
// - time evolution
//    psi(r,t+dt) = exp(-i dt H) psi(r,t),  (H:Hamiltonian of the system)
//      H = -delta/2 + V(r), delta = d^2/dx^2
//    psi(r,t+dt) = exp(-i dt H) psi(r,t)
//                ~ {exp(-i(dt/2)V} {exp(i dt(delta/2)} {exp(-i(dt/2)V} psi(r,t)
//
//      {exp(-i(dt/2)V} psi(r,t): phase rotate theta = (dt/2)*V
//
//      {exp(i dt(delta/2)}: Richardson algorithm - explecit method
//        Richardson; Computer Physics Communications vol.63 page 84-94 (1991)
//
// --------------------
*/

const electronPQM2D = (function(){ // ====================  electronPQM2D Module  ====================

	// au: atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;			// (m) 1(au) = g_auLength (m)
	const g_auTime = 2.418884326e-17;			// (s) 1(au) = g_auTime (s)
	const g_auEnergy = 4.35974465e-18;			// (J) 1(au) = g_auEnergy (J)
	const g_au2eV = 27.211386;					// (eV) 1(au) = 27.211386 (eV)
	const g_nxMax = 320;						// g_NNx max
	const g_nyMax = 320;						// g_NNy max
	const g_nMax = 320;							// = Math.max(g_nxMax,g_nyMax);

	let g_NNx = 160;							// number of space x-division, g_NNx*g_dx == x-boxsize
	let g_NNy = 160;							// number of space y-division, g_NNy*g_dy == y-boxsize
	let g_sysTime = 0.0;						// (au) system time
	let g_dx = 0.5;								// (au) x-division
	let g_dy = 0.5;								// (au) y-division
	let g_timeStep = 0.25*g_dx*g_dx;			// (au) time step dt=0.5*g_dx*g_dx
	let g_dampingFactor = 0.05;					// for lossEnergy() :steepest descent method

	const g_psi = dim3( g_nxMax, g_nyMax, 2 );	// g_psi[g_nMax][g_nMax][2] wave function
												// g_psi[][][0]:real part g_psi[][][][1]:imaginary part
	const g_vv = dim2( g_nxMax, g_nyMax );		// g_vv[g_nMax][g_nMax] external potential
	const g_wrk = dim3( g_nxMax, g_nyMax, 2 );	// g_wrk[g_nMax][g_nMax][2] work space in lossEnergy()

	const g_srnd = dim1( 1002 );				// 1000 RND orderd series 0 to 1,use drawCloud()
	const g_cloud = dimInt2( g_nxMax, g_nyMax );// if g_cloud[i][j]>0, plot cloud point

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {
		const xMax=g_NNx*g_dx, yMax=g_NNy*g_dy;

		g_sysTime = 0.0;
		if (theme==0) { // 0:free space
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setWallPotential(0.5*xMax, 5*g_dx, 0.0); //(xPos, vThick, vHeight)
		} else if (theme==1) { // 1:parabolic potential
			setGaussianWave(xMax/2.0,yMax/4.0,3.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setParabolicPotential(2.0); // v(x)=k0*(x-x0)^2, k0=2.0
		}  else if (theme==2) { // 2:tunnel effect
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky) K=0.5
			setWallPotential(0.5*xMax, 5*g_dx, 0.55); //(xPos, vThick, vHeight)
		}  else if (theme==3) { // 3:step hill
			setGaussianWave(0.25*xMax,0.25*yMax,5.0,1.0,1.0); //(xPos,yPos,waveWidth,kx,ky)
			setWallPotential(0.5*xMax, xMax, 0.4); //(xPos, vThick, vHeight)
		}  else if (theme==4) { // 4:double slit
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setSlitPotential(0.5*xMax,5*g_dx,4.0,0.05*yMax,0.10*yMax); //(Pos,Thick,Height,slitWidth,slitSpan)
		}  else if (theme==5) { // 5:cylinder
			setGaussianWave(0.25*xMax,0.5*yMax,5.0,1.0,0.0); //(xPos,yPos,waveWidth,kx,ky)
			setCylinderPotential(0.6*xMax,0.4*yMax,0.1*xMax,1.0); //(xPos,yPos, radius, vHeight)
		}
	}

	function setGaussianWave(xPos,yPos,waveWidth,kx,ky) {
		const nnx=g_NNx, nny=g_NNy, psi=g_psi;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const x = i*g_dx, y = j*g_dy;
				const phAb = Math.exp(-((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos))/(4*waveWidth*waveWidth) );
				const phPh = kx*x+ky*y;
				psi[i][j][0] = phAb*Math.cos(phPh);
				psi[i][j][1] = phAb*Math.sin(phPh);
			}
		}
		for (let i=0; i<nnx; i++) {
			psi[i][0][0] = 0.0;
			psi[i][0][1] = 0.0;
			psi[i][nny-1][0] = 0.0;
			psi[i][nny-1][1] = 0.0;
		}
		for (let j=0; j<nny; j++) {
			psi[0][j][0] = 0.0;
			psi[0][j][1] = 0.0;
			psi[nnx-1][j][0] = 0.0;
			psi[nnx-1][j][1] = 0.0;
		}
		normalize(psi);
	}

	function setParabolicPotential(k0) { // V(r)= k0*r^2
		const nnx=g_NNx, nny=g_NNy, x0=nnx*g_dx/2.0, y0 = nny*g_dy/2.0;

		const aa = k0/(nnx*g_dx*nnx*g_dx/4.0);
		for (let i=0; i<nnx; i++) {
			const x = i*g_dx
			for (let j=0; j<nny; j++) {
				const y = j*g_dy;
				g_vv[i][j] = aa*((x-x0)*(x-x0)+(y-y0)*(y-y0));
			}
		}
	}

	function setWallPotential(xPos, vThick, vHeight) {
		const nnx=g_NNx, nny=g_NNy;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const x = i*g_dx, y = j*g_dy;
				g_vv[i][j] = (x>=xPos && x<xPos+vThick) ? vHeight : 0.0;
			}
		}
	}

	function setCylinderPotential(xPos,yPos, radius, vHeight) {
		const nnx=g_NNx, nny=g_NNy;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const x = i*g_dx-xPos, y = j*g_dy-yPos;
				g_vv[i][j] = (x*x+y*y < radius*radius) ? vHeight : 0.0;
			}
		}
	}

	function setSlitPotential(wallPos, wallThick, wallHeight, slitWidth, slitSpan) {
		const nnx=g_NNx, nny=g_NNy;

		const ym = nny*g_dy/2.0;
		const w = slitWidth/2.0;
		const d = slitSpan/2.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const x = i*g_dx, y = j*g_dy;
				if (x>=wallPos && x<wallPos+wallThick) {
					g_vv[i][j] = wallHeight;
					if ((y>=ym-d-w && y<=ym-d+w) || (y>=ym+d-w && y<=ym+d+w)) g_vv[i][j] = 0.0;
				} else {
					g_vv[i][j] = 0.0;
				}
			}
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution(lossSW) {

		for (let i=0; i<4; i++) {
			g_sysTime += g_timeStep;
			phaseStep(g_psi,g_vv,0.5*g_timeStep);
			kxStep(g_psi,g_timeStep);
			kyStep(g_psi,g_timeStep);
			phaseStep(g_psi,g_vv,0.5*g_timeStep);
		}
		if (lossSW==1) lossEnergy(g_psi,g_vv,g_dampingFactor);
	}

	function phaseStep(ph, vv, dt) {
		const nnx=g_NNx, nny=g_NNy;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const th = dt*vv[i][j];
				const cs = Math.cos(th), sn = Math.sin(th);
				const phr = ph[i][j][0];
				const phi = ph[i][j][1];
				ph[i][j][0] = cs*phr+sn*phi;
				ph[i][j][1] = cs*phi-sn*phr;
			}
		}
	}

	function kxStep(ph, deltat) {
		const nnx=g_NNx, nny=g_NNy;
		const epsilonX = deltat/(g_dx*g_dx);
		const alphaXRe = 0.5 * (1.0+Math.cos(epsilonX/2));
		const alphaXIm = -0.5 * Math.sin(epsilonX/2);
		const betaXRe = Math.sin(epsilonX/4)*Math.sin(epsilonX/4);
		const betaXIm = 0.5 * Math.sin(epsilonX/2);

		for (let j=0; j<nny; j++) {
			for (let i=0; i<nnx-1; i+=2) {
				substepX(ph,i,i+1,j);
			}
			for (let i=1; i<nnx-1; i+=2) {
				substepX(ph,i,i+1,j);
			}
			substepX(ph,nnx-1,0,j); // periodic-x condition

			substepX(ph,nnx-1,0,j); // periodic-x condition
			for (let i=1; i<nnx-1; i+=2) {
				substepX(ph,i,i+1,j);
			}
			for (let i=0; i<nnx-1; i+=2) {
				substepX(ph,i,i+1,j);
			}
		}

		function substepX(ph,i,i2,j) {
			const xRe = ph[i][j][0], xIm = ph[i][j][1];
			const yRe = ph[i2][j][0], yIm = ph[i2][j][1];
			ph[i][j][0] = (alphaXRe*xRe-alphaXIm*xIm) + (betaXRe*yRe-betaXIm*yIm);
			ph[i][j][1] = (alphaXRe*xIm+alphaXIm*xRe) + (betaXRe*yIm+betaXIm*yRe);
			ph[i2][j][0] = (alphaXRe*yRe-alphaXIm*yIm) + (betaXRe*xRe-betaXIm*xIm);
			ph[i2][j][1] = (alphaXRe*yIm+alphaXIm*yRe) + (betaXRe*xIm+betaXIm*xRe);
		}
	}

	function kyStep(ph, deltat) {
		const nnx=g_NNx, nny=g_NNy;
		const epsilonY = deltat/(g_dy*g_dy);
		const alphaYRe = 0.5 * (1.0+Math.cos(epsilonY/2));
		const alphaYIm = -0.5 * Math.sin(epsilonY/2);
		const betaYRe = Math.sin(epsilonY/4)*Math.sin(epsilonY/4);
		const betaYIm = 0.5 * Math.sin(epsilonY/2);

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny-1; j+=2) {
				substepY(ph,i,j,j+1);
			}
			for (let j=1; j<nny-1; j+=2) {
				substepY(ph,i,j,j+1);
			}
			substepY(ph,i,nny-1,0); // periodic-y condition

			substepY(ph,i,nny-1,0); // periodic-y condition
			for (let j=1; j<nny-1; j+=2) {
				substepY(ph,i,j,j+1);
			}
			for (let j=0; j<nny-1; j+=2) {
				substepY(ph,i,j,j+1);
			}
		}

		function substepY(ph, i,j,j2) {
			const xRe = ph[i][j][0], xIm = ph[i][j][1];
			const yRe = ph[i][j2][0], yIm = ph[i][j2][1];
			ph[i][j][0] = (alphaYRe*xRe-alphaYIm*xIm) + (betaYRe*yRe-betaYIm*yIm);
			ph[i][j][1] = (alphaYRe*xIm+alphaYIm*xRe) + (betaYRe*yIm+betaYIm*yRe);
			ph[i][j2][0] = (alphaYRe*yRe-alphaYIm*yIm) + (betaYRe*xRe-betaYIm*xIm);
			ph[i][j2][1] = (alphaYRe*yIm+alphaYIm*yRe) + (betaYRe*xIm+betaYIm*xRe);
		}
	}

	function lossEnergy(ph, vv, damp) {
		const nnx=g_NNx, nny=g_NNy, h2=2.0*g_dx*g_dx, wrk=g_wrk;
		const ee = meanKinetic(ph) + meanPotential(ph,vv);
		for (let i=0; i<nnx; i++) {
			const ipp = (i+1)%nnx, imm = (i-1+nnx)%nnx;
			for (let j=0; j<nny; j++) {
				const jpp = (j+1)%nny, jmm = (j-1+nny)%nny;
				wrk[i][j][0] = -(ph[ipp][j][0]+ph[imm][j][0]+ph[i][jpp][0]+ph[i][jmm][0]-4.0*ph[i][j][0])/h2
								+(vv[i][j]-ee)*ph[i][j][0];
				wrk[i][j][1] = -(ph[ipp][j][1]+ph[imm][j][1]+ph[i][jpp][1]+ph[i][jmm][1]-4.0*ph[i][j][1])/h2
								+(vv[i][j]-ee)*ph[i][j][1];
			}
		}
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				ph[i][j][0] -= damp*wrk[i][j][0];
				ph[i][j][1] -= damp*wrk[i][j][1];
			}
		}
		normalize(ph);
	}


	// --------------------  utility  --------------------

	function norm(ph) {
		const nnx=g_NNx, nny=g_NNy;
		let p = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				p += (ph[i][j][0]*ph[i][j][0]+ph[i][j][1]*ph[i][j][1]);
			}
		}
		return p*g_dx*g_dy;
	}

	function normalize(ph) {
		const nnx=g_NNx, nny=g_NNy;
		const a = 1.0/Math.sqrt(norm(ph));
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				ph[i][j][0] *= a;
				ph[i][j][1] *= a;
			}
		}
	}

	function meanPotential(ph, vv) {
		const nnx=g_NNx, nny=g_NNy;
		let p = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				p += vv[i][j]*(ph[i][j][0]*ph[i][j][0]+ph[i][j][1]*ph[i][j][1]);
			}
		}
		return p*g_dx*g_dy;
	}

	function meanKinetic(ph) {
		const nnx=g_NNx, nny=g_NNy, h2=g_dx*g_dx;
		let p=0.0;

		for (let i=0; i<nnx; i++) {
			const ipp = (i+1)%nnx, imm = (i-1+nnx)%nnx;
			for (let j=0; j<nny; j++) {
				const jpp = (j+1)%nny, jmm = (j-1+nny)%nny;
				const d2phRe = (ph[ipp][j][0]+ph[imm][j][0]+ph[i][jpp][0]+ph[i][jmm][0]-4.0*ph[i][j][0])/h2;
				const d2phIm = (ph[ipp][j][1]+ph[imm][j][1]+ph[i][jpp][1]+ph[i][jmm][1]-4.0*ph[i][j][1])/h2;
				p += (ph[i][j][0]*d2phRe+ph[i][j][1]*d2phIm);
			}
		}
		return -0.5*p*g_dx*g_dy;
	}

	function psiDensity(i,j) {
		return (g_psi[i][j][0]*g_psi[i][j][0]+g_psi[i][j][1]*g_psi[i][j][1]);
	}

	function psiPhase(i,j) { // 0...2PI
		return (Math.PI + Math.atan2(g_psi[i][j][1],g_psi[i][j][0])); // atan2(y,x)
	}

	function psiXCurrent(i,j) {
		const ip = (i+1)%g_NNx;
		const im = (i-1+g_NNx)%g_NNx;
		const pRe = (g_psi[ip][j][1]-g_psi[im][j][1])/(2*g_dx);
		const pIm = (-g_psi[ip][j][0]+g_psi[im][j][0])/(2*g_dx);
		return (g_psi[i][j][0]*pRe + g_psi[i][j][1]*pIm)*g_dx*g_dy;
	}

	function psiYCurrent(i,j) {
		const jp = (j+1)%g_NNy;
		const jm=(j-1+g_NNy)%g_NNy;
		const pRe = (g_psi[i][jp][1]-g_psi[i][jm][1])/(2*g_dy);
		const pIm = (-g_psi[i][jp][0]+g_psi[i][jm][0])/(2*g_dy);
		return (g_psi[i][j][0]*pRe + g_psi[i][j][1]*pIm)*g_dx*g_dy;
	}

	// ---  cloud  ---

	function setCloud() {
		const nnx=g_NNx, nny=g_NNy;

		set_srnd();
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_cloud[i][j] = 0;
			}
		}
		let s = 0, ip = 0
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_cloud[i][j] = 0;
				s += psiDensity(i,j)*g_dx*g_dy;
				while (s>g_srnd[ip] && ip<1000) {
					g_cloud[i][j] += 1;
					ip += 1;
				}
			}
		}
	}

	function set_srnd() {
		g_srnd[0] = Math.random();
		for (let i=1; i<1001; i++) {
			g_srnd[i] = g_srnd[i-1] + Math.random();
		}
		for (let i=0; i<1000; i++) {
			g_srnd[i] = g_srnd[i]/g_srnd[1000];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition(theme,canvasId)
		evolve:			timeEvolution,			// timeEvolution(lossSW)

		setCloud:		setCloud,				// setCloud()

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_NNy, g_dx, g_timeStep ]; },
		getNow:			function() { return [ g_sysTime,norm(g_psi),meanKinetic(g_psi),meanPotential(g_psi,g_vv) ]; },
		getPsiDensity:	psiDensity,				// psiDensity(i,j)
		getPsiPhase:	psiPhase,				// psiPhase(i,j)
		getPsiCurrent:	function(i,j) { return [ psiXCurrent(i,j), psiYCurrent(i,j) ]; },
		getPsi:			function(i,j) { return [ g_psi[i][j][0], g_psi[i][j][1] ]; },
		getVext:		function(i,j) { return g_vv[i][j]; },
		getCloud:		function(i,j) { return g_cloud[i][j]; },
	}

})(); // ====================  electronPQM2D end  ====================


const js110 = (function(){ // ====================  js Module  ====================

	const theModule = electronPQM2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');
	let imgVext = null;			// image in draw()

	let v_theme = 1;			// 0:free space, 1:parabolic potential, 2:tunnel effect 3:step hill 4:step higher hill
	let v_lossSW = 0;			// if lossSW==1, call lossEnergy()

	let p_auLength, p_auTime, p_auEnergy, p_au2eV; // <-- theModule.getAUinSI();
	let p_NNx, p_NNy, p_dx, p_timeStep; // <-- theModule.getSysParam();
	let p_NNz;

  let sysTime, norm, meanKinetic, meanPotential;
	let nowData = [];
  let vextArray = [];
  let psiReArray = [];
  let psiImArray = [];

	let dispMode = 9;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let onlyFlag = false;

  let breakFlag = false;
  let getFieldFlag = true;
  let fieldKind = 1;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics - mouse event listener
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme ); // ( nn, BoxSizeInNM, contTemp )
			imgVext = null;
			[ p_auLength, p_auTime, p_auEnergy, p_au2eV ] = theModule.getAUinSI();
			[ p_NNx, p_NNy, p_dx, p_timeStep ] = theModule.getSysParam();
			p_NNz = p_NNx;
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift )
			g3d.init( p_NNx, p_NNy, p_NNz, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
			g3d.drawVectorField2D.threshold = 0.2;
      getFieldFlag = true;
      fieldKind = 1;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_lossSW );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			onlyFlag = true;
			theModule.evolve( v_lossSW );
		}

		draw( ctx, dispMode );

    if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    if (fieldKind==1) {
      vextArray = [];
      psiReArray = [];
      psiImArray = [];
      nowData = [ sysTime, norm, meanKinetic, meanPotential ];
      for (let i=0; i<p_NNx; i++) {
        vextArray[i] = [];
        psiReArray[i] = [];
        psiImArray[i] = [];
        for (let j=0; j<p_NNy; j++) {
          vextArray[i][j] = theModule.getVext(i,j);
          let pRe, pIm;
          [ pRe, pIm ] = theModule.getPsi(i,j);
          psiReArray[i][j] = pRe;
          psiImArray[i][j] = pIm;
        }
      }
    }
  }


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const xp = 80, yp = 20, sc = 2, xBoxSize = 320, yBoxSise = 320, yTextPos = yCanvasSize-45; // image2d
		const colour = { text:'#888888', box:'#999900', vext:'#00ff00' };

		ctx.clearRect( 0, 0, xCanvasSize, yCanvasSize );

		if ( dispMode<=3 ) {
			ctx.strokeStyle = colour.box;
			ctx.strokeRect( xp, yp, xBoxSize, yBoxSise );

			if ( imgVext == null ) setImg_Vext( ctx, p_NNx, p_NNy, xp, yp, sc );
			ctx.putImageData( imgVext, xp, yp ); // potential vv[][] imageData
		}

		if ( dispMode==0 || dispMode==1 || dispMode==2 ) { // 0:density 1:phase 2:prob.current
			drawPh( ctx, dispMode, p_NNx, p_NNy, xp, yp, sc );
		} else if ( dispMode==3 ) { // 3:cloud
			drawCloud( ctx, p_NNx, p_NNy, xp, yp, sc );

		} else if (dispMode==4 ) { // 4: 3D Vext(x,y)
			const zFunc = function(i,j) { return 10.0*theModule.getVext(i,j); };
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, "#00aa00", 2 );
		} else if (dispMode==5 ) { // 4: 3D density(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j); };
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, 1.0, 2 );
		} else if (dispMode==6 ) { // 4: 3D Re[psi](x,y)
			const zFunc = function(i,j) { return 150.0*theModule.getPsi(i,j)[0]; };
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, 1.0, 1 );
		} else if (dispMode==7 ) { // 7: 3D phase(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j)};
			const colorFunc = function(i,j) {
				const hue = theModule.getPsiPhase(i,j)*180.0/Math.PI;
				const z = 1500.0*theModule.getPsiDensity(i,j);
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? "hsl("+hue+",100%,50%)" : "#444444";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );
		} else if (dispMode==8 ) { // 8: 3D (density+Vext)(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j) + 10.0*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const zc = 1500.0*theModule.getPsiDensity(i,j)
				const th = (18120 - Math.floor(180.0*zc/g3d.cz0))%360;
				return ( Math.abs(zc/g3d.cz0)>0.005 ) ? "hsl("+(th)+",100%,50%)" : "hsl("+(th)+",30%,20%)";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );
		} else if (dispMode==9 ) { // 9: 3D (phase+Vext)(x,y)
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j) + 10.0*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const hue = theModule.getPsiPhase(i,j)*180.0/Math.PI;
				const z = 1500.0*theModule.getPsiDensity(i,j);
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? "hsl("+(hue)+",100%,50%)" : "#004400";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );
		} else if (dispMode==10 ) { // 10: 3D (density+Vext)(x,y) + prob.current
			const zFunc = function(i,j) { return 1500.0*theModule.getPsiDensity(i,j) + 10.0*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const zc = 1500.0*theModule.getPsiDensity(i,j)
				const th = (18120 - Math.floor(180.0*zc/g3d.cz0))%360;
				return ( Math.abs(zc/g3d.cz0)>0.005 ) ? "hsl("+(th)+",100%,50%)" : "hsl("+(th)+",30%,20%)";
			}
			//g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 2 );

			const vFunc = function(i,j) {
				const mag = 15000.0;
				const z = 1500.0*theModule.getPsiDensity(i,j)
				if ( z<0.01 ) return [ 0, 0 ];
				let cx,cy;
				[ cx, cy ] = theModule.getPsiCurrent(i,j);
				return [ cx*mag, cy*mag ];
			}
			//drawVectorField2D(ctx, rotAngle, zFunc, vFunc, colorMode, inc [, showBox] )
			g3d.drawVectorField2D(ctx, 0.0, zFunc, vFunc, 1, 4, 2 )
		}

		[ sysTime, norm, meanKinetic, meanPotential ] = theModule.getNow();
		const totalEnergy = meanKinetic + meanPotential;
		ctx.fillStyle = colour.text;
		ctx.fillText(`time = ${sysTime} (au)`, 20, yTextPos);
		ctx.fillText(`norm = ${norm.toFixed(15)}`, 240, yTextPos);
		ctx.fillText(`kineticE = ${(meanKinetic*p_au2eV).toFixed(2)} (eV)`, 20, yTextPos+20);
		ctx.fillText(`potentialE = ${(meanPotential*p_au2eV).toFixed(2)} (eV)`, 240, yTextPos+20);
		ctx.fillText(`totalE= ${(totalEnergy*p_au2eV).toFixed(2)} (eV)`, 20, yTextPos+40);
		ctx.fillText("box = 80x80 (au)", 240, yTextPos+40);

	}

	function setImg_Vext( ctx, nnx, nny, xp, yp, sc ) {
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const vij = theModule.getVext(i,j);
				const g = Math.min(Math.floor(vij*50),255);
				ctx.fillStyle = "rgb(0,"+g+",0)";
				ctx.fillRect( i*sc+xp, (nny-j-1)*sc+yp, sc, sc );
			}
		}
		imgVext = ctx.getImageData(xp,yp,320,320);
	}

	function drawPh( ctx, drawMode, nnx, nny, xp, yp, sc ) {
		const velocityMag = 600000.0;

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const d = theModule.getPsiDensity(i,j)*100000;
				const vij = theModule.getVext(i,j);
				const g = Math.min(Math.floor(vij*50),255);
				if (d>20.0) {
					if (drawMode==0 ||drawMode==2) { // 0:density 2:prob.current
						const a = Math.min(Math.floor(d),255);
						ctx.fillStyle = `rgb(${a},${g},${a})`;
					} else if (drawMode==1) { // 1:phase
						const p = theModule.getPsiPhase(i,j)*180.0/Math.PI;
						const a = 0.2*Math.min(Math.floor(d),255);
						ctx.fillStyle = `hsl(${p},100%,${a}%)`;
					}
					ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
				}
			}
		}
		if (drawMode==2) { // 2:prob.current, draw current lines
			for (let i=1; i<nnx-1; i+=4) {
				for (let j=1; j<nny-1; j+=4) {
					const d = theModule.getPsiDensity(i,j)*100000;
					if (d>20.0) {
						let cxij, cyij;
						[ cxij, cyij ] = theModule.getPsiCurrent(i,j)
						const cx = cxij*p_timeStep*velocityMag;
						const cy = -cyij*p_timeStep*velocityMag;
						const col = (cx>0) ? "rgb(80,80,250)" : "rgb(250,80,80)";
						drawLine( ctx, i*sc+xp, (nny-j-1)*sc+yp, i*sc+cx+xp, (nny-j-1)*sc+cy+yp, col );
					}
				}
			}
		}
	}

	function drawCloud( ctx, nnx, nny, xp, yp, sc ) { // drawMode = 3:cloud

		if ( !pauseFlag || onlyFlag ) {
			theModule.setCloud();
			onlyFlag = false;
		}
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				const cij = theModule.getCloud(i,j)
				if ( cij>0 ) {
					const vij = theModule.getVext(i,j);
					const g = Math.min(Math.floor(vij*50),255);
					const p = Math.min( cij, 5 ) ;
					ctx.fillStyle = "hsl("+(75-p*15)+",100%,50%)";
					ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
				}
			}
		}
	}

	function drawLine( ctx, x1, y1, x2, y2, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension grid2d  created 2023.06.01, last updated 2023.06.04
	// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
	g3d.drawGrid2D = function ( ctx, rotAngle, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawGrid2D.threshold = 0.005;

	// g3d_extension gvextor field  created 2023.06.01, last updated 2023.06.08
	// g3d.drawVectorField2D(ctx, rotAngle, zFunc, vFunc, colorMode, inc [, showBox] )
	g3d.drawVectorField2D = function(ctx, rotAngle, zFunc, vFunc, colorMode, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, threshold = g3d.drawVectorField2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj+=inc) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;

				const f = zFunc(i,nny-j-1);
				const x = i*dx, y = j*dx, z = f + g3d.cz0;
				let vx, vy;
				[ vx, vy ] = vFunc(i,nny-j-1);
				if ( vx*vx+vy*vy < threshold ) continue;
				const x1 = x + vx, y1 = y -vy, z1 = z;
				let colr;
				if ( colorMode==0 ) {
					const hue = Math.floor((Math.atan2(vy,vx))*180.0/Math.PI+180.0);
					colr = `hsl(${hue},100%,50%)`;
				} else if ( colorMode==1 ) {
					colr = (vx>0) ? "#3333ff" : "#ff3333";
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawVectorField2D.threshold = 0.5;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setLossSW() {  // select lossSW
		v_lossSW = 0 + document.getElementById("slct_loss").selectedIndex;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle( 15, -10 );
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, norm, meanKinetic, meanPotential ];
  }

  function pygetFieldData() {
		fieldKind = 0;
    return [ nowData, vextArray, psiReArray, psiImArray ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setLossSW:		setLossSW,		// setLossSW()
		setDispMode:	setDispMode,	// setDispMode()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, norm, kineticEnergy, potentialEnergy ]
    pygetFieldData: pygetFieldData, // pygetFieldData() : return [ nowData, vextArray, psiReArray, psiImArray ]
	};

})(); // ====================  js110 module end  ====================


const js = js110;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js110] electrons - Periodic Quantum Mechanics 2D (explecit method)</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>in free box</option><option selected>in parabollic V=k r^2</option>
<option>tunnel effect</option><option>step hill</option>
<option>double slit</option><option>cylinder scattering</option>
</select>
    <span style="margin-right: 10px;"></span>
<label>loss:</label>
<select id="slct_loss" onChange="js.setLossSW()">
<option selected>off</option><option>on</option></select>
    <span style="margin-right: 20px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>2D: psi density</option><option>2D: phase (angle:Im(psi)/Re(psi))</option>
<option>2D: probability current</option><option>2D electron cloud</option>
<option>grid2D: Vext(x,y)</option><option>grid2D: density(x,y)</option>
<option>grid2D: Re[psi(x,y)]</option><option>grid2D: phase(x,y)</option>
<option>grid2D: (density + Vext)(x,y)</option><option selected>grid2D: (phase + Vext)(x,y)</option>
<option>grid2D: prob.current +(density + Vext)(x,y)</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(f'i = {i:>2},  time = {sysTime:>8.2f} (au),  energy = {energy:>10.6f} (au)')
  time.sleep(1)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change theme, dispMode

import time

themeList = [
  '0: in free box', '1: in parabollic V=k r^2', '2: tunnel effect', '3: step hill',
  '4: double slit', '5: cylinder scattering' ]

dispModeList = [
  '0: 2D - psi density', '1: 2D - phase (angle:Im(psi)/Re(psi))', '2: 2D - probability current','3: 2D - electron cloud',
  '4: grid2D - Vext(x,y)', '5: grid2D - density(x,y)', '6: grid2D - Re[psi(x,y)]', '7: grid2D - phase(x,y)',
  '8: grid2D - (density + Vext)(x,y)', '9: grid2D - (phase + Vext)(x,y)', '10: grid2D - prob.current +(density + Vext)(x,y)' ]

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
for theme in [ 0, 1, 4 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print("-- set theme :", themeList[theme], " --")

  # chabge disp mode and get data and print
  for dispMode in [ 0, 1, 5, 9 ]:
    # change dispMode
    print(f"  -- dispMode: {dispModeList[dispMode]} --")
    eval_js( 'js.pysetDispMode({})'.format(dispMode) )

    # get data and print
    [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
    energy = kineticEnergy + potentialEnergy
    print(f'\t time = {sysTime:>8.2f} (au),  energy = {energy:>10.6f} (au)')
    time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# total energy = K + U ~ constant

import time
import matplotlib.pyplot as plt

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
theme = 1 # '1: in parabollic V=k r^2'
eval_js( 'js.pysetTheme({})'.format(theme) )
print("-- set theme :", themeList[theme], " --")

# change dispMode
dispMode = 1 # '1: 2D - phase (angle:Im(psi)/Re(psi))'
print(f"  -- dispMode: {dispModeList[dispMode]} --")
eval_js( 'js.pysetDispMode({})'.format(dispMode) )

# get K U data
timeList = []
kkList = []
uuList = []
energyList = []

for i in range(50):
  [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  if i%5==0:
    print(f"\t time = {sysTime:>8.2f} (au),  energy(K+U) = {energy:>8.6f} (au),  K = {kineticEnergy:>8.6f} (au),  U = {potentialEnergy:>8.6f} (au)")
  timeList.append(sysTime)
  kkList.append(kineticEnergy)
  uuList.append(potentialEnergy)
  energyList.append(energy)
  time.sleep(1)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

print("-- plot energy(blue), kinetic energy(red), potential energy(green) --")
plt.plot( timeList, kkList, 'or-' ) # kinetic energy red
plt.plot( timeList, uuList, '^g-' ) # potential energy green
plt.plot( timeList, energyList, 'xb-' ) # total energy blue
plt.xlabel("time (au)")
plt.ylabel("energy (au)")
plt.show()

In [ ]:
# get data vextList, psiReList, psiImList and save

import time
import numpy as np

themeList = [
  '0: in free box', '1: in parabollic V=k r^2', '2: tunnel effect', '3: step hill',
  '4: double slit', '5: cylinder scattering' ]

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
theme = 1 # '1: in parabollic V=k r^2'
eval_js( 'js.pysetTheme({})'.format(theme) )
print("-- set theme :", themeList[theme], " --")

# get data and print
for i in range(10):
  [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print(f'i = {i:>2},  time = {sysTime:>8.2f} (au),  energy = {energy:>10.6f} (au)')
  time.sleep(1)

# get field data
print("-- get field data --")
[ nowData, vextArray, psiReArray, psiImArray ] = eval_js( 'js.pygetFieldData()' )
[ sysTime, norm, kineticEnergy, potentialEnergy ] = nowData
energy = kineticEnergy + potentialEnergy
print(f' time stamp = {sysTime:>8.2f} (au),  energy = {energy:>10.6f} (au)')

# set numpy array
Vext = np.array(vextArray)
PsiRe = np.array(psiReArray)
PsiIm = np.array(psiImArray)
print(f' Vext shape :{Vext.shape},  PsiRe shape :{PsiRe.shape},  PsiIm shape :{PsiIm.shape}')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# save field data
print("-- save data : np.array([ vextArray, psiReArray, psiImArray ]) --")
np_data = np.array([ vextArray, psiReArray, psiImArray ])

np.save('js110_data.npy', np_data)

print("-- saved data shape :", np_data.shape, " --")

In [ ]:
# load data and set numpy array for plot

import numpy as np

# load data : loaded_data <--  np.array([ vextArray, psiReArray, psiImArray ])
loaded_data = np.load('js110_data.npy')

# set numpy array
Vext = loaded_data[0]            # Vext(x,y)
PsiRe = loaded_data[1]           # Re[Psi(x,y)]
PsiIm = loaded_data[2]           # Im[Psi(x,y)]
Dens = PsiRe**2 + PsiIm**2       # Psi(x,y) density
Phase = np.arctan2(PsiIm, PsiRe) # Psi(x,y) phase
print(f' Vext shape :{Vext.shape},  PsiRe shape :{PsiRe.shape},  PsiIm shape :{PsiIm.shape}')
print(f' Dens shape :{Dens.shape},  Phase shape :{Phase.shape}')

In [ ]:
# wave function Re[psi(x,y)] / image plot

import numpy as np
import matplotlib.pyplot as plt

def image_field(fld,titleStr):
  Z = fld.T
  img = plt.imshow(Z, origin='lower', cmap='jet' )
  plt.colorbar(img)
  plt.title(titleStr)
  plt.show()

# PsiRe image
image_field(PsiRe, "wave function Re[psi(x,y)] image")

In [ ]:
# Vext, Dens, phase image

image_field(Vext, "external potential Vext(x,y) image")
image_field(Dens, "electron density(x,y) image")
image_field(Phase*Dens, "wave function phase(x,y) image")

In [ ]:
# plot  Vext(x), Psi(x) - plotly

import plotly.graph_objects as go
import numpy as np

# create grid data
nx, ny = PsiRe.shape
X = np.arange(0, nx)
Y = np.arange(0, ny)
X, Y = np.meshgrid(X, Y)

# plot Re[Psi(x)]
Z = PsiRe.T
fig = go.Figure(data=go.Surface(x=X, y=Y, z=Z, name='Psi(x)', opacity=0.8))

# plot Vext(x)
Z = Vext.T * 0.05
fig.add_trace(go.Surface(x=X, y=Y, z=Z, name='Vext(x)', opacity=0.2))

fig.update_layout(
    title='Psi(x) and Vext(x)',
    width=1000, height=700,
    scene=dict(
        xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
        aspectmode="manual",
        aspectratio=dict(x=1, y=ny/nx, z=0.5),  # set aspect ratio
        camera=dict(
            eye=dict(x=0.0, y=-1.5, z=0.7),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=0, z=1)  # upside vector
        )
    )
)

fig.show()

# save the figure as a html file
#fig.write_html("js110_plot_Psi_Vext.html")

In [ ]:
# prompt: calc norm of Psi(x,y) = PsiRe + i PsiIm,  norm = sum of | Psi |^2, dx= dy = 0.5

# Assuming PsiRe, PsiIm, and the dx, dy values are already defined as in your provided code.

def calculate_norm(PsiRe, PsiIm, dx, dy):
  """Calculates the norm of the wave function Psi(x,y).

  Args:
    PsiRe: A NumPy array representing the real part of Psi(x,y).
    PsiIm: A NumPy array representing the imaginary part of Psi(x,y).
    dx: The step size in the x-direction.
    dy: The step size in the y-direction.

  Returns:
    The calculated norm of Psi(x,y).
  """
  norm = np.sum(np.abs(PsiRe + 1j * PsiIm)**2) * dx * dy
  return norm

# Example usage (replace with your actual data)
# Assuming you have already loaded your data into PsiRe, PsiIm as shown in the example
# loaded_data = np.load('js110_data.npy')
# PsiRe = loaded_data[1]
# PsiIm = loaded_data[2]

dx = 0.5
dy = 0.5

norm_value = calculate_norm(PsiRe, PsiIm, dx, dy)
print(f"The norm of Psi(x,y) is: {norm_value}")